In [1]:
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

In [8]:
dadosFreq = pd.read_excel("Dados originais_artigo Priscila .xlsx")

In [9]:
dadosFreq.head()

,Espécie - Abelha,Freq. 1 (Hz),Freq. 2 (Hz),Freq. 3 (Hz),Média Freq.,Peso abelha (g),Tamanho tórax,Duração (s),Média Freq..1,Peso abelha (g).1,Tamanho tórax.1,Duração (s).1
0,Augochloropsis_sp1,219.4,197.3,203.7,206.800000,0.0608,2.1,4.1780,15.62052,0.007294,0.122474,1.570395
1,Augochloropsis_sp1,206.4,205.1,216.0,209.166667,0.0608,2.1,1.6670,NaN,NaN,NaN,NaN
2,Augochloropsis_sp1,187.3,186.6,204.0,192.633333,0.0448,1.9,0.5803,NaN,NaN,NaN,NaN
3,Augochloropsis_sp1,234.7,229.3,214.8,226.266667,0.0534,1.9,0.7520,NaN,NaN,NaN,NaN
4,Augochloropsis_sp1,214.6,194.8,205.8,205.066667,0.0448,1.9,NaN,NaN,NaN,NaN,NaN


In [10]:
dados = dadosFreq.drop(["Peso abelha (g)", "Tamanho tórax", "Duração (s)", "Média Freq..1",\
                            "Peso abelha (g).1", "Tamanho tórax.1", "Duração (s).1"], axis=1)

In [11]:
dados.head()

,Espécie - Abelha,Freq. 1 (Hz),Freq. 2 (Hz),Freq. 3 (Hz),Média Freq.
0,Augochloropsis_sp1,219.4,197.3,203.7,206.800000
1,Augochloropsis_sp1,206.4,205.1,216.0,209.166667
2,Augochloropsis_sp1,187.3,186.6,204.0,192.633333
3,Augochloropsis_sp1,234.7,229.3,214.8,226.266667
4,Augochloropsis_sp1,214.6,194.8,205.8,205.066667


In [12]:
dados['Espécie - Abelha'] = dados['Espécie - Abelha'].apply(lambda x: x.split('_')[0])
dados.head()

,Espécie - Abelha,Freq. 1 (Hz),Freq. 2 (Hz),Freq. 3 (Hz),Média Freq.
0,Augochloropsis,219.4,197.3,203.7,206.800000
1,Augochloropsis,206.4,205.1,216.0,209.166667
2,Augochloropsis,187.3,186.6,204.0,192.633333
3,Augochloropsis,234.7,229.3,214.8,226.266667
4,Augochloropsis,214.6,194.8,205.8,205.066667


In [13]:
X = list(dados['Média Freq.'])
y = list(dados['Espécie - Abelha'])

In [14]:
for i in range(len(X)):
    X[i] = [X[i]]
print(X)

[[206.8], [209.166666666667], [192.633333333333], [226.266666666667], [205.066666666667], [180.3], [190.1], [296.466666666667], [207.9], [257.1], [232.566666666667], [214.133333333333], [227.433333333333], [264.1], [208.533333333333], [235.3], [208.3], [330.80808080808], [312.033333333333], [327.933333333333], [196.733333333333], [188.233333333333], [175.933333333333], [118.733333333333], [175.866666666667], [158.6], [175.9], [169.866666666667], [158.3], [303.666666666667], [351.833333333333], [341.6], [318.7], [371.7], [366.433333333333], [298.433333333333], [345.2], [324.6], [330.6], [314.833333333333], [322.266666666667], [250.666666666667], [243.833333333333], [279.3], [221.033333333333], [204.266666666667], [220.033333333333], [207.2], [223.766666666667], [250.866666666667], [214.166666666667], [190.133333333333], [219.2], [205.433333333333], [182.433333333333], [153.333333333333], [153.8], [193.366666666667], [148.3], [136.333333333333], [147.166666666667]]


In [15]:
def salva_modelo(modelo, nome_arq):
    filename = '/home/alison/Documentos/Projeto-Tomate/modelos_gen/' + nome_arq
    pickle.dump(modelo, open(filename, 'wb'))


In [16]:
def treino_teste(matriz, classes, clf):
    X_train, X_test, y_train, y_test = train_test_split(matriz, classes, test_size=0.5, random_state=1)
    #kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    clf.fit(X_train, y_train)

    y_true, y_pred = y_test, clf.predict(X_test)
    
    print("Acurácia...: %.4f" %(metrics.accuracy_score(y_true, y_pred) * 100))
    print("Precision..: %.4f" %(metrics.precision_score(y_true, y_pred, average='macro') * 100))
    print("Recall.....: %.4f" %(metrics.recall_score(y_true, y_pred, average='macro') * 100))
    print("F1-Score...: %.4f" %(metrics.f1_score(y_true, y_pred, average='macro') * 100))
    print()
    print(metrics.classification_report(y_true, y_pred))

    return clf

In [17]:
def algoritmos(op, matriz, classes):

    if op == "svm":
        modelo = SVC(C=10, decision_function_shape='ovo', gamma=0.01, kernel='rbf')
        
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_svm_.sav"
        salva_modelo(clf, modelname)

    elif op == "lr":
        modelo = LogisticRegression(C=10)
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_lr_.sav"
        salva_modelo(clf, modelname)

    elif op == "dtree":
        modelo = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_split=2)
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_dtree_.sav"
        salva_modelo(clf, modelname)

    elif op == "rf":
        modelo = RandomForestClassifier(max_features='sqrt', n_estimators=100)
        clf = treino_teste(matriz, classes, modelo)

        modelname = "baseline_rf_.sav"
        salva_modelo(modelo, modelname)

    elif op == "ens":
        svc = SVC(C=10)
        rf = RandomForestClassifier()
        lr = LogisticRegression(C=1)

        modelos = [('svc', svc), ('rf', rf), ('lr', lr)]

        votingclf = VotingClassifier(estimators=modelos, voting='hard')
        clf = treino_teste(matriz, classes, votingclf)

        modelname = "baseline_ensemble_.sav"
        salva_modelo(clf, modelname)

In [18]:
algoritmo = "svm"
algoritmos(algoritmo, X, y)

Acurácia...: 48.3871
Precision..: 28.5714
Recall.....: 29.2411
F1-Score...: 27.6488

                       precision    recall  f1-score   support

       Augochloropsis       0.17      0.50      0.25         2
               Bombus       0.57      0.50      0.53         8
              Centris       0.00      0.00      0.00         2
              Eulaema       0.00      0.00      0.00         0
          Exomalopsis       0.83      0.62      0.71         8
             Melipona       0.71      0.71      0.71         7
Pseudoalglochloropsis       0.00      0.00      0.00         2
             Xylocopa       0.00      0.00      0.00         2

             accuracy                           0.48        31
            macro avg       0.29      0.29      0.28        31
         weighted avg       0.53      0.48      0.50        31



In [18]:
algoritmo = "lr"
algoritmos(algoritmo, X, y)

Acurácia...: 16.1290
Precision..: 12.8571
Recall.....: 9.4388
F1-Score...: 10.8844

                       precision    recall  f1-score   support

       Augochloropsis       0.00      0.00      0.00         2
               Bombus       0.00      0.00      0.00         8
              Centris       0.00      0.00      0.00         2
          Exomalopsis       0.50      0.38      0.43         8
             Melipona       0.40      0.29      0.33         7
Pseudoalglochloropsis       0.00      0.00      0.00         2
             Xylocopa       0.00      0.00      0.00         2

             accuracy                           0.16        31
            macro avg       0.13      0.09      0.11        31
         weighted avg       0.22      0.16      0.19        31



In [19]:
algoritmo = "rf"
algoritmos(algoritmo, X, y)

Acurácia...: 16.1290
Precision..: 7.4603
Recall.....: 9.4388
F1-Score...: 8.3333

                       precision    recall  f1-score   support

       Augochloropsis       0.00      0.00      0.00         2
               Bombus       0.00      0.00      0.00         8
              Centris       0.00      0.00      0.00         2
          Exomalopsis       0.30      0.38      0.33         8
             Melipona       0.22      0.29      0.25         7
Pseudoalglochloropsis       0.00      0.00      0.00         2
             Xylocopa       0.00      0.00      0.00         2

             accuracy                           0.16        31
            macro avg       0.07      0.09      0.08        31
         weighted avg       0.13      0.16      0.14        31



In [20]:
algoritmo = "dtree"
algoritmos(algoritmo, X, y)

Acurácia...: 12.9032
Precision..: 6.3492
Recall.....: 7.6531
F1-Score...: 6.9328

                       precision    recall  f1-score   support

       Augochloropsis       0.00      0.00      0.00         2
               Bombus       0.00      0.00      0.00         8
              Centris       0.00      0.00      0.00         2
          Exomalopsis       0.22      0.25      0.24         8
             Melipona       0.22      0.29      0.25         7
Pseudoalglochloropsis       0.00      0.00      0.00         2
             Xylocopa       0.00      0.00      0.00         2

             accuracy                           0.13        31
            macro avg       0.06      0.08      0.07        31
         weighted avg       0.11      0.13      0.12        31



In [21]:
algoritmo = "ens"
algoritmos(algoritmo, X, y)

Acurácia...: 19.3548
Precision..: 9.1575
Recall.....: 11.2245
F1-Score...: 9.8378

                       precision    recall  f1-score   support

       Augochloropsis       0.00      0.00      0.00         2
               Bombus       0.00      0.00      0.00         8
              Centris       0.00      0.00      0.00         2
          Exomalopsis       0.31      0.50      0.38         8
             Melipona       0.33      0.29      0.31         7
Pseudoalglochloropsis       0.00      0.00      0.00         2
             Xylocopa       0.00      0.00      0.00         2

             accuracy                           0.19        31
            macro avg       0.09      0.11      0.10        31
         weighted avg       0.15      0.19      0.17        31

